In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=4)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [4]:
# Agent State is a dictionary that contains the messages that the agent has received
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        # Initialize the graph with AgentState
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        # When we have don all the setups and turn it in a langchain runnable
        self.graph = graph.compile()
        # Attribute into a dictionary to map name of the tool to the tool object
        self.tools = {t.name: t for t in tools}
        # Passing in list of tools to the model letting it know what tools are available to call
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4-turbo")
abot = Agent(model, [tool], system=prompt)

In [10]:
from IPython.display import Image

Image(abot.graph.get_graph().draw_png())

ImportError: DLL load failed while importing _graphviz: The specified module could not be found.

In [11]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_IfovJOlHh0TrPkSiric0ZYDO'}
Back to the model!


In [12]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IfovJOlHh0TrPkSiric0ZYDO', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 153, 'total_tokens': 175}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_9b82693875', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-102a54a2-88d8-4c6c-a4d1-867fc239fc44-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_IfovJOlHh0TrPkSiric0ZYDO'}]),
  ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_ep

In [13]:
result['messages'][-1].content

'The current weather in San Francisco is sunny with a temperature of 64°F (17.8°C). The wind is blowing from the west-northwest at 12.5 mph (20.2 kph), and the humidity is at 65%. The conditions are clear with no precipitation, and visibility is around 8 miles (14 kilometers).'

In [14]:
# Note that the results will happen in parallel
# The weather in SF and LA will be fetched at the same time as they are independent and not related
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_Hkl7qAppoUCso5QrFPvwvV4I'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_fjyS2pshaKyRQkhf7d0juABG'}
Back to the model!


In [15]:
result['messages'][-1].content

'The current weather is as follows:\n\n- **San Francisco**: \n  - Temperature: 55.9°F (13.3°C)\n  - Conditions: Partly cloudy\n  - Humidity: 87%\n  - Wind: NNW at 3.8 mph (6.1 kph)\n\n- **Los Angeles**:\n  - Temperature: 61.0°F (16.1°C)\n  - Conditions: Overcast\n  - Humidity: 90%\n  - Wind: N at 2.2 mph (3.6 kph)'

In [16]:
# Note that the results will happen sequentially this time. 
# It is because it actually needs the result of the first query to make the second query
query = "Who won the super bowl in 2024? What is the GDP of state where the winning team is located?" 
messages = [HumanMessage(content=query)]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': '2024 Super Bowl winner'}, 'id': 'call_AE3GmC26FUSF63Nb4NhJCSMb'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Missouri 2023'}, 'id': 'call_Oe21A5aIaIMETzklS5Kpuc0e'}
Back to the model!


In [17]:
result['messages'][-1].content

'The Kansas City Chiefs won the Super Bowl in 2024.\n\nThe Gross Domestic Product (GDP) of Missouri, the state where the Kansas City Chiefs are located, was approximately $344.12 billion in 2023.'

In [18]:
print(result['messages'])

[HumanMessage(content='Who won the super bowl in 2024? What is the GDP of state where the winning team is located?'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AE3GmC26FUSF63Nb4NhJCSMb', 'function': {'arguments': '{"query":"2024 Super Bowl winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 169, 'total_tokens': 191}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_31a6c0accf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3d4fcbc2-1cac-4b3d-9471-2b21ac958671-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Super Bowl winner'}, 'id': 'call_AE3GmC26FUSF63Nb4NhJCSMb'}]), ToolMessage(content='[{\'url\': \'https://people.com/super-bowl-2024-full-recap-8572245\', \'content\': \'"\\nTo make those dreams come true, an "absolute army of people" took the field to set up the elaborate stage as soon as the last play of the sec

: 